In [29]:
import streamlit as st
from langchain_community.llms import Ollama
from retriever import load_hybrid_retriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [30]:
load_dotenv()  # take environment variables from .env file
# llm = Ollama(model="llama3", base_url="http://localhost:11434")
from langchain_community.llms import HuggingFaceHub
import os

# Make sure to set this in Hugging Face Spaces Secrets
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B",  # You can swap with another model
    huggingfacehub_api_token=hf_token,
    temperature=0.3,
    max_new_tokens=512
)


# Load prebuilt retriever
retriever = load_hybrid_retriever()

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a legal AI assistant. Use the given context from IPC/Constitution to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer concisely, but accurately in legal terms.
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [31]:
query = "what is the punishment for murder under IPC?"

In [32]:
result = qa_chain({"query": query})

In [33]:
result

{'query': 'what is the punishment for murder under IPC?', 'result': ''}

In [34]:
for doc in result.get("source_documents", []):
        st.write(f"- {doc.metadata.get('source', 'Unknown')}")